<h1 style="text-align: center;">
    Trabajo Práctico
</h1>

<h2 style="text-align: center;">
    Sistemas Automáticos de Diagnóstico y Detección de Fallas II
</h2>

<h3 style="text-align: center;">
    Alumnos
</h3>

<div style="display: flex;justify-content: center;align-items: center">
<table>
  <tbody>
    <tr>
      <th><h5 style="text-align: center;">Nombre</h5></th>
      <th><h5 style="text-align: center;">Padrón</h5></th>
    </tr>
    <tr>
      <td><h5 style="text-align: center;">Barreneche Franco</h5></td>
      <td><h5 style="text-align: center;">102205</h5></td>
    </tr>
    <tr>
      <td><h5 style="text-align: center;">Botta Guido</h5></td>
      <td><h5 style="text-align: center;">102103</h5></td>
    </tr>
    <tr>
      <td><h5 style="text-align: center;">Carol Lugones Ignacio</h5></td>
      <td><h5 style="text-align: center;">100073</h5></td>
    </tr>
    <tr>
      <td><h5 style="text-align: center;">Pernin Alejandro</h5></td>
      <td><h5 style="text-align: center;">92216</h5></td>
    </tr>
  </tbody>
</table>
</div>

# Importación de bibliotecas

In [ ]:
import numpy as np
import pandas as pd
from copy import copy
import random
import os                                                 
import sys
import dowhy
from dowhy import CausalModel
module_path = os.path.abspath(os.path.join('../server'))                                                      
if module_path not in sys.path:                        
    sys.path.append(module_path)                    
from expertSystem.expertSystem import getCandidateBeers
import itertools

# Generación de DataSet sintético

In [ ]:
INTENSITY_TYPES = ["baja", "media", "alta", "*"]
COLOR_TYPES = ["palido", "ambar", "oscuro", "*"]
BITTERNESS_TYPES = ["bajo", "medio", "alto", "*"]
HOP_TYPES = ["viejo mundo", "nuevo mundo", "*"]
FERMENTATION_TYPES = ["baja", "media", "alta", "*"]
YEAST_TYPES = ["lager", "ale", "*"]

In [ ]:
NUM_CLIENTS = 8000

feature_types = {
    'intensity': INTENSITY_TYPES,
    'color': COLOR_TYPES,
    'bitterness': BITTERNESS_TYPES,
    'hop': HOP_TYPES,
    'fermentation': FERMENTATION_TYPES,
    'yeast': YEAST_TYPES,
}
template = {
    'intensity': '*',
    'color': '*',
    'bitterness': '*',
    'hop': '*',
    'fermentation': '*',
    'yeast': '*'
}
features = list(template.keys())

responses = []
for client_idx in range(NUM_CLIENTS):
    response = copy(template)
    random.shuffle(features)
    for feature in features:
        response[feature] = random.choice(feature_types[feature])
        engine_result = getCandidateBeers(response)
        candidates = engine_result['candidateBeers']
        if not candidates:
            response['beer'] = 'None'
            responses.append(response)
            break
        if len(candidates) == 1:
            response['beer'] = candidates[0]
            responses.append(response)
            break

result_df = pd.DataFrame(responses)
result_df.head(10)

In [ ]:
allOptions = list(itertools.product(*feature_types.values()))
responsesI = []

for x in allOptions:
    d = {
        'intensity': x[0],
        'color': x[1],
        'bitterness': x[2],
        'hop': x[3],
        'fermentation': x[4],
        'yeast': x[5]
    }
    r = getCandidateBeers(d)['candidateBeers']
    if len(r) > 2:
        continue
    if len(r) == 1:
        d['beer'] = r[0]
    else:
        d['beer'] = 'None'
    responsesI.append(d)

In [ ]:
result_df = pd.DataFrame(responsesI)
result_df.head(10)

# Análisis de inferencia causal

In [ ]:
intensity_mapping = {"*":0,"baja":1,"media":2,"alta":3}
color_mapping = {'*': 0, 'palido': 1, 'oscuro': 2, 'ambar': 3}
bitterness_mapping = {"*":0,"bajo":1,"medio":2,"alto":3}
hop_mapping ={"*": 0, "viejo mundo" : 1, "nuevo mundo": 2}
fermentation_mapping = intensity_mapping
yeast_mapping = {"*":0, "lager":1, "ale":2}
beer_mapping = {'Cream Ale': 0, 'None': 1,'Kolsch': 2, 'Lager Ambar Checa': 3, 'Baltic Porter': 4, 'Ipa Blanca': 5}

result_df = result_df.replace({
    'intensity': intensity_mapping,
    'color': color_mapping,
    'bitterness': bitterness_mapping,
    'hop': hop_mapping,
    'fermentation': fermentation_mapping,
    'yeast': yeast_mapping,
    'beer': beer_mapping
})
result_df

In [ ]:
def generate_causal_model(target_feature):
    target_df = result_df.copy()
    target_df['treatment'] = result_df[target_feature] != 0
    causal_graph = """digraph {
                        hop;
                        fermentation;
                        yeast;
                        
                        intensity;
                        color;
                        bitterness;
    
                        treatment;
    
                        hop->beer;
                        fermentation->beer;
                        yeast->beer;
                        intensity->beer;
                        color->beer;
                        bitterness->beer;
                        treatment->beer;
                        Z->treatment;
                   """ 
    causal_graph = causal_graph + target_feature + "->treatment;}"
    model = dowhy.CausalModel(data=target_df,
                              graph=causal_graph.replace("\n", " "),
                              treatment="treatment",
                              outcome="beer")
    #model.view_model()
    return model

In [ ]:
def refute_model(model):
    identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
    estimate = model.estimate_effect(identified_estimand,
                                     method_name='backdoor.propensity_score_matching',
                                     target_units='att')
    refutation = model.refute_estimate(identified_estimand, 
                                       estimate, 
                                       method_name='placebo_treatment_refuter',
                                       placebo_type='permute', 
                                       num_simulations=20)
    return refutation

## Features invididuales

In [ ]:
p_values = dict()
for feature in features:
    print(feature.upper())
    model = generate_causal_model(feature)
    refutation = refute_model(model)
    p_values[feature] = refutation.refutation_result['p_value']
    print(refutation)

In [ ]:
feature = 'yeast'
print(feature.upper())
model = generate_causal_model(feature)
refutation = refute_model(model)
print(refutation)

In [ ]:
print(p_values)

In [ ]:
weights = dict()
total = 0
for feature, p_value in p_values.items():
    weight = 1 - p_value
    weights[feature] = weight
    total += weight

for feature in weights.keys():
    weights[feature] = weights[feature]/total

print(weights)

In [ ]:
x = weights
l = {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}
for k in l.keys():
    print(k, l[k])

## Features jerarquizadas

In [ ]:
target_df = result_df.copy()
target_df['treatment'] = (target_df['intensity'] != 0) & (target_df['fermentation'] != 0) & (target_df['color'] != 0)
causal_graph = """digraph {
                    hop;
                    fermentation;
                    yeast;
                    
                    intensity;
                    color;
                    bitterness;

                    treatment;

                    intensity->beer;
                    color->beer;
                    bitterness->beer;
                    
                    Z->treatment;
                    hop->intensity;
                    yeast->intensity;
                    fermentation->intensity;
                    
                    hop->color;
                    yeast->color;
                    fermentation->color;
                    
                    hop->bitterness;
                    yeast->bitterness;
                    fermentation->bitterness;
                    
                    hop->treatment;
                    yeast->treatment;
                    fermentation->treatment;
                    
                    treatment->bitterness;
                    treatment->color;
                    treatment->intensity;
                }
               """ 
model = dowhy.CausalModel(data=target_df,
                          graph=causal_graph.replace("\n", " "),
                          treatment="treatment",
                          outcome="beer")
model.view_model()
print(refute_model(model))

# IA Explainable

In [ ]:
from pycaret.datasets import get_data
from pycaret.classification import *
data = result_df
clf1 = setup(data, target = 'beer', session_id=123)
#best_model = compare_models()
dt = create_model('dt', cross_validation=False)
plot_model(dt)

In [ ]:
plot_model(dt, plot = 'feature')

In [ ]:
interpret_model(dt)

In [ ]:
interpret_model(dt, plot = 'correlation')